In [1]:
import boto3
from dotenv import load_dotenv
import os
import json
import json_repair
from langfuse import Langfuse
load_dotenv(".env")

True

In [2]:
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY= os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_REGION = os.getenv("AWS_REGION")

client = boto3.client('bedrock-runtime',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION
)


In [3]:
LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY")
LANGFUSE_HOST = os.getenv("LANGFUSE_HOST")
langfuse = Langfuse(
    public_key=LANGFUSE_PUBLIC_KEY,
    secret_key=LANGFUSE_SECRET_KEY,
    host=LANGFUSE_HOST
)

In [4]:
response = client.count_tokens(
    modelId="anthropic.claude-sonnet-4-20250514-v1:0",
    input={
        "converse": {
            "messages": [
                {"role": "user", "content": [{"text": "Xin chào Claude"}]}
            ],
            # "toolConfig": {
            #     "tools": [
            #         {
            #             "toolSpec": {
            #                 "name": "my_tool",
            #                 "description": "Mô tả tool",
            #                 "inputSchema": { ... }
            #             }
            #         }
            #     ]
            # }
        }
    }
)
print("Estimated input tokens:", response['inputTokens'])


Estimated input tokens: 13


In [5]:
class TokenCounter:
    def __init__(self, model_id, aws_access_key_id, aws_secret_access_key, region_name):
        self.model_id = model_id
        self.client = boto3.client('bedrock-runtime',
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
            region_name=region_name
        )

    def count_tokens(self, system, messages):
        response = self.client.count_tokens(
            modelId=self.model_id,
            input={
                "converse": {
                    "messages": messages,
                    "system": system
                }
            }
        )
        return response['inputTokens'], response['outputTokens']

    # def create_message(self, role, text, image: None, document: None, toolUse: None, toolResult: None, ):


In [ ]:
session_id = "cd388252-6004-4fe1-b3eb-4113e0e986a2"
result = {"session_id": session_id, "traces": [
    {
        "trace_id": trace['id'],
        "observations": []
    }
]}

# Lấy thông tin session
print(f"Session id: {session_id}")
session = langfuse.api.sessions.get(session_id)
session_json = json.loads(session.json())

# Lấy trace id
traces = session_json['traces']
traces = sorted(traces, key=lambda x: x["timestamp"])
for trace in traces:
    trace_result = {
        "trace_id": trace['id'],
        "observations": []
    }
    print(f"\tGetting observation of trace id: {trace['id']}")
    # Lấy thông tin trace
    trace = langfuse.api.trace.get(trace['id'])
    trace_json = json.loads(trace.json())
    observations = trace_json['observations']
    observations = sorted([obs for obs in observations if obs['type'] == 'GENERATION'], key=lambda x: x["startTime"])
    for obs in observations:
        obs_result = {
            "observation_id": obs['id'],
            "messages": [],
            "system": []
        }
        print(f"\t\tObservation id: {obs['id']}")
        obs_input = obs['input']
        system_temp = {}
        message_temp = {}
        for inp in obs_input:
            if 'role' not in inp.keys():
                if 'additional_kwargs' not in inp.keys():
                    system_temp["text"] = inp["content"]
                else:
                    message_temp['role'] = "user"
                    if 'function' in inp['additional_kwargs']:
                        message_temp['content'] = {
                            "toolUse": {
                                "toolUseId": inp['additional_kwargs']['tool_calls'][0]['id'],
                                "name": inp['additional_kwargs']['tool_calls'][0]['name'],
                            }
                        }
            else:
                if 

                



        


Session id: cd388252-6004-4fe1-b3eb-4113e0e986a2
	Getting observation of trace id: 60cd5de5-eeb1-4773-8f9e-3ecce36868bd
		Observation id: 14b6cc3a-461f-4bf1-a841-fb88930a703e
	Getting observation of trace id: 93affe42-25cf-4cef-99ff-c3d700ce21d6
		Observation id: 86cc668a-6316-46c1-aaab-0d48ba5e894b
		Observation id: 89a85722-d2eb-4ef2-afbe-bd33c62762e8
	Getting observation of trace id: bf478e59-8999-4bf9-bb68-d28ad1c39a64
		Observation id: 9b809252-a36e-4e21-9f57-862cf29cb536
		Observation id: 2ee380eb-3088-45e7-95ac-92d677aac452
		Observation id: 9b46f7cd-3192-4c23-bcd5-2929575c9bca


In [30]:
obs['input']

[{'content': '## ROLE\nBạn là **Trợ lý tìm kiếm và phân tích thông tin** .  \nNhiệm vụ của bạn là **hỗ trợ tra cứu thông tin bên ngoài** từ các nguồn đáng tin cậy, phục vụ cho các mục tiêu như:\n- Nghiên cứu chuyên sâu\n- Đối chiếu thị trường\n- Cập nhật công nghệ\n- Theo dõi xu hướng ngành\n- Tin tức các loại\n\n## INSTRUCTIONS\n\n### Step 1: Xác định ngôn ngữ người dùng\n\n- Trong một số trường hợp, đầu vào có thể bao gồm:\n  - Nội dung từ tệp tin do người dùng tải lên (phần đầu)\n  - Câu hỏi chính của người dùng nằm sau dòng có tiền tố `USER QUERY:`\n\n- **Nếu phát hiện dòng có tiền tố `USER QUERY:`**, hãy:\n  - **Chỉ sử dụng nội dung sau dòng `USER QUERY:`** để xác định ngôn ngữ và xử lý các bước tiếp theo.\n  - Bỏ qua toàn bộ phần nội dung phía trên (nội dung file hoặc chú thích).\n\n- Nếu **không có `USER QUERY:`**, sử dụng toàn bộ đầu vào để xác định ngôn ngữ.\n\n- Chỉ hỗ trợ các ngôn ngữ sau:\n  - Tiếng Việt\n  - Tiếng Anh\n  - Tiếng Nhật\n  - Tiếng Trung (Giản thể hoặc Phồn th

In [43]:
response = client.count_tokens(
    modelId="anthropic.claude-sonnet-4-20250514-v1:0",  # ví dụ model
    input={
        "converse": {
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"text": "Hãy tính tổng 3 + 4 giúp tôi."}
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {
                            "toolUse": {
                                "toolUseId": "jlkjlkjkljklkljlksdasdasdasdasdaa",
                                "name": "calculator",
                                "input": {"operation": "add", "numbers": [3, 4]}
                            }
                        }
                    ]
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "toolResult": {
                                "toolUseId": "jlkjlkjkljklkljlksdasdasdasdasdaa",
                                "content": [
                                    {"json": {"result": 7}}
                                ],
                                "status": "success"
                            }
                        }
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {"text": "Kết quả là 7."}
                    ]
                }
            ],
            "system": [
                {"text": "Bạn là một trợ lý AI hữu ích."}
            ]
        }
    }
)

print(response)


{'ResponseMetadata': {'RequestId': 'e77367ea-87b9-4a76-bc42-af4cd9c4f43d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 25 Aug 2025 07:48:46 GMT', 'content-type': 'application/json', 'content-length': '19', 'connection': 'keep-alive', 'x-amzn-requestid': 'e77367ea-87b9-4a76-bc42-af4cd9c4f43d'}, 'RetryAttempts': 0}, 'inputTokens': 147}
